In [1]:
using CSV
using DataFrames

using JuMP
using Gurobi

using LinearAlgebra
using Statistics
using Random
Random.seed!(42)  # For reproducibility

TaskLocalRNG()

## Import data

In [ ]:
# Paths (relative to notebook structure)
restaurant_path    = "../clean_data/restaurant_data_expanded.csv"
scrap_path         = "../clean_data/food_scrap_locations.csv"
neighborhood_path  = "../clean_data/neighborhood_supply.csv"

# Read CSVs into DataFrames
food_scrap_locations  = CSV.read(scrap_path, DataFrame)
neighborhood_supply   = CSV.read(neighborhood_path, DataFrame)
restaurant_data       = CSV.read(restaurant_path, DataFrame)
restaurant_data = restaurant_data[shuffle(1:nrow(restaurant_data))[1:1000], :]

# Verify
println("Sampled restaurants: ", nrow(restaurant_data))
println("Total supply: ", sum(restaurant_data.waste) / 1e6, " million lbs")

Sampled restaurants: 1000
Total supply: 75.08486887683786 million lbs


# Clean the data:

## Ensure no commas in numbers, each field cast as correct data type, drop unnecessary columns, and no negative supply from synthetic data

In [3]:
# ================================
# 3. CLEAN RESTAURANT
# ================================
# Columns: latitude | longitude | waste
rename!(restaurant_data, names(restaurant_data)[3] => :supply)

# Ensure Float64
restaurant_data.supply    = Float64.(restaurant_data.supply)
restaurant_data.latitude  = Float64.(restaurant_data.latitude)
restaurant_data.longitude = Float64.(restaurant_data.longitude)

# *** DROP RESTAURANTS WITH NEGATIVE SUPPLY ***
filter!(row -> row.supply >= 0, restaurant_data)



# ================================
# 3. CLEAN FOOD SCRAP CENTER DATA
# ================================
# rename Latitude and Longitude to latitude and longitude for consistency
rename!(food_scrap_locations, names(food_scrap_locations)[15] => :latitude)
rename!(food_scrap_locations, names(food_scrap_locations)[16] => :longitude)

# Keep only coordinates we need
food_scrap_locations.latitude = Float64.(food_scrap_locations.latitude)
food_scrap_locations.longitude = Float64.(food_scrap_locations.longitude)

# Keep only coordinate columns in food_scrap_locations
select!(food_scrap_locations, [:latitude, :longitude])

# ================================
# 4. CLEAN NEIGHBORHOOD SUPPLY DATA
# ================================
# Rename demand column for clarity
rename!(neighborhood_supply, names(neighborhood_supply)[4] => :supply_gap)

# Now neighborhood_supply.supply_gap might be String OR Float64.
# Only do replace/parse if it's strings.
if eltype(neighborhood_supply.supply_gap) <: AbstractString
    neighborhood_supply.supply_gap =
        parse.(Float64, replace.(neighborhood_supply.supply_gap, "," => ""))
end

# Demand = positive deficit, surplus -> 0
neighborhood_supply.demand = max.(0.0, -neighborhood_supply.supply_gap)

neighborhood_supply.latitude  = Float64.(neighborhood_supply.latitude)
neighborhood_supply.longitude = Float64.(neighborhood_supply.longitude)

# keep only necessary columns from neighborhood supply
select!(neighborhood_supply, [:latitude, :longitude, :demand, :Year])

# ================================
# 5. SHOW CLEANED HEADS
# ================================
println("=== Restaurants (cleaned) ===")
println(first(restaurant_data, 5))

println("\n=== Food Scrap Locations (cleaned) ===")
println(first(food_scrap_locations, 5))

println("\n=== Neighborhood Supply (cleaned) ===")
println(first(neighborhood_supply, 5))

=== Restaurants (cleaned) ===
5×3 DataFrame
 Row │ latitude  longitude  supply        
     │ Float64   Float64    Float64       
─────┼────────────────────────────────────
   1 │  40.7424   -73.9943      1.11117e5
   2 │  40.7323   -73.8722  71532.9
   3 │  40.7794   -73.9492  98561.5
   4 │  40.6102   -73.9208  19103.5
   5 │  40.7315   -73.861   94690.4

=== Food Scrap Locations (cleaned) ===
5×2 DataFrame
 Row │ latitude  longitude 
     │ Float64   Float64   
─────┼─────────────────────
   1 │  40.6355   -74.0228
   2 │  40.7526   -73.969
   3 │  40.7635   -74.0002
   4 │  40.762    -73.9693
   5 │  40.7174   -74.0108

=== Neighborhood Supply (cleaned) ===
5×4 DataFrame
 Row │ latitude  longitude  demand     Year  
     │ Float64   Float64    Float64    Int64 
─────┼───────────────────────────────────────
   1 │  40.8267   -73.9217  1.02143e5   2025
   2 │  40.8321   -73.8887  3.33493e5   2025
   3 │  40.8016   -73.9374  0.0         2025
   4 │  40.8469   -73.8918  1.13653e5   202

## Get vectors for supply[i] for all restaurants, demand[k] for all neighborhoods, and cij[i,j] and cjk[j,k] to plug directly into JuMP model

In [4]:
# ================
# 6. EXTRACT VECTORS
# ================
R = nrow(restaurant_data)
D = nrow(food_scrap_locations)
N = nrow(neighborhood_supply)

supply = restaurant_data.supply              # s_i
demand = neighborhood_supply.demand          # d_k

# ================
# 7. MANHATTAN DISTANCE FUNCTION
# ================
manhattan(lat1, lon1, lat2, lon2) = abs(lat1 - lat2) + abs(lon1 - lon2)

# ================
# 8. COST MATRICES
# ================
# cij: Restaurants (i) → Donation centers (j)
cij = [manhattan(restaurant_data.latitude[i], restaurant_data.longitude[i],
                 food_scrap_locations.latitude[j], food_scrap_locations.longitude[j])
       for i in 1:R, j in 1:D]

# cjk: Donation centers (j) → Neighborhoods (k)
cjk = [manhattan(food_scrap_locations.latitude[j], food_scrap_locations.longitude[j],
                 neighborhood_supply.latitude[k], neighborhood_supply.longitude[k])
       for j in 1:D, k in 1:N]

println("Size of cij (R x D): ", size(cij))
println("Size of cjk (D x N): ", size(cjk))


Size of cij (R x D): (1000, 201)
Size of cjk (D x N): (201, 591)


# Sanity check data

In [5]:
println("Demand: min = ", minimum(demand), ", max = ", maximum(demand))
println("Any NaN in demand? ", any(isnan.(demand)))
println("Any Inf in demand? ", any(isinf.(demand)))

println("Any NaN in cij? ", any(isnan.(cij)))
println("Any Inf in cij? ", any(isinf.(cij)))

println("Any NaN in cjk? ", any(isnan.(cjk)))
println("Any Inf in cjk? ", any(isinf.(cjk)))

Demand: min = 0.0, max = 4.27814874096381e6
Any NaN in demand? false
Any Inf in demand? false
Any NaN in cij? false
Any Inf in cij? false
Any NaN in cjk? false
Any Inf in cjk? false


In [6]:
R = nrow(restaurant_data)
D = nrow(food_scrap_locations)
N = nrow(neighborhood_supply)

println("R, D, N = ", (R, D, N))

R, D, N = (1000, 201, 591)


In [7]:
println("Size of cij: ", size(cij))  # should be (R, D)
println("Size of cjk: ", size(cjk))  # should be (D, N)

Size of cij: (1000, 201)
Size of cjk: (201, 591)


# Cost-Reduction Optimization Model 

We formulate the minimum-cost redistribution problem below.

**Sets**

- $R$: restaurants (supply nodes)  
- $D$: food scrap / donation centers (transshipment nodes)  
- $N$: neighborhoods (demand nodes)

**Parameters**

- $s_i$: supply at restaurant $i \in R$  
- $d_k$: demand at neighborhood $k \in N$  
- $c_{ij}$: cost of transporting one unit from restaurant $i$ to donation center $j$  
- $c_{jk}$: cost of transporting one unit from donation center $j$ to neighborhood $k$  
- $M$: large penalty coefficient for unmet demand

**Decision Variables**

- $x_{ij} \ge 0$: shipment from restaurant $i$ to donation center $j$  
- $y_{jk} \ge 0$: shipment from donation center $j$ to neighborhood $k$  
- $u_k \ge 0$: unmet demand at neighborhood $k$

---

### **Objective: Minimize Total Cost**

$ \displaystyle
\min\;
\sum_{i \in R} \sum_{j \in D} c_{ij} x_{ij}
\;+\;
\sum_{j \in D} \sum_{k \in N} c_{jk} y_{jk}
\;+\;
M \sum_{k \in N} u_k
$

---

### **Constraints**

**1. Restaurant supply limits**

$ \displaystyle
\sum_{j \in D} x_{ij} \le s_i \quad \forall i \in R
$

**2. Neighborhood demand balance**

$ \displaystyle
\sum_{j \in D} y_{jk} + u_k = d_k \quad \forall k \in N
$

**3. Flow conservation at donation centers**

$ \displaystyle
\sum_{k \in N} y_{jk}
=
\sum_{i \in R} x_{ij}
\quad \forall j \in D
$

**4. Nonnegativity**

$ \displaystyle x_{ij},\; y_{jk},\; u_k \ge 0 $


In [8]:
println("Any supply < 0? ", any(supply .< 0))
println("Any demand < 0? ", any(demand .< 0))
println("Any NaN in supply? ", any(isnan.(supply)))
println("Any NaN in demand? ", any(isnan.(demand)))


Any supply < 0? false
Any demand < 0? false
Any NaN in supply? false
Any NaN in demand? false


In [9]:
# ================================
# 9. REDUCED COST OPTIMIZATION MODEL
# ================================

# Big-M penalty for unmet demand
# Here we pick M as the total demand so that leaving demand unmet is very expensive
M = sum(demand)

model_cost = Model(Gurobi.Optimizer)

set_silent(model_cost)

# Decision variables:
# x[i,j] = flow of food from restaurant i to donation center j
# y[j,k] = flow of food from donation center j to neighborhood k
# u[k]   = unmet demand at neighborhood k
@variable(model_cost, x[1:R, 1:D] >= 0)
@variable(model_cost, y[1:D, 1:N] >= 0)
@variable(model_cost, u[1:N] >= 0)

# Objective:
# Minimize transportation cost + big-M penalty on unmet demand
@objective(model_cost, Min,
    sum(cij[i,j] * x[i,j] for i in 1:R, j in 1:D) +
    sum(cjk[j,k] * y[j,k] for j in 1:D, k in 1:N) +
    M * sum(u[k] for k in 1:N)
)

# Constraints:

# 1) Restaurant supply: cannot ship more than available surplus
@constraint(model_cost, [i in 1:R],
    sum(x[i,j] for j in 1:D) <= supply[i]
)

# 2) Neighborhood demand balance:
#    inflow from centers + unmet demand = demand
@constraint(model_cost, [k in 1:N],
    sum(y[j,k] for j in 1:D) + u[k] == demand[k]
)

# 3) Donation centers are pure transshipment nodes:
#    total inflow from restaurants = total outflow to neighborhoods
@constraint(model_cost, [j in 1:D],
    sum(y[j,k] for k in 1:N) == sum(x[i,j] for i in 1:R)
)

# ================================
# 10. SOLVE AND INSPECT
# ================================
optimize!(model_cost)

total_demand   = sum(demand)
total_received = sum(value.(y))   # sum over all j,k
total_unmet    = sum(value.(u))   # sum over all k

reduction_abs = total_received
reduction_pct = 100 * reduction_abs / total_demand

println("Termination status: ", termination_status(model_cost))
println("Objective value (total cost + penalty): ", objective_value(model_cost))
println("Total neighborhood demand: ", total_demand)
println("Total received by neighborhoods: ", total_received)
println("Total unmet demand: ", total_unmet)
println("Demand reduction (absolute): ", reduction_abs)
println("Demand reduction (%): ", reduction_pct, "%")

# After optimize!(model_cost)

# Per-neighborhood received under COST model
r_cost = [sum(value.(y[j, k]) for j in 1:D) for k in 1:N]

println("=== Neighborhoods That Received Food ===\n")

count_nonzero_cost = 0
for k in 1:N
    if r_cost[k] > 0
        count_nonzero_cost += 1
        println("Neighborhood $(k): received $(round(r_cost[k], digits=2)) pounds")
    end
end

println("\nTotal neighborhoods receiving any food: $count_nonzero_cost")


Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Termination status: OPTIMAL
Objective value (total cost + penalty): 1.179186228254354e16
Total neighborhood demand: 1.5243933021550444e8
Total received by neighborhoods: 7.508486887683785e7
Total unmet demand: 7.73544613386666e7
Demand reduction (absolute): 7.508486887683785e7
Demand reduction (%): 49.25557516599548%
=== Neighborhoods That Received Food ===

Neighborhood 1: received 102142.62 pounds
Neighborhood 2: received 333492.65 pounds
Neighborhood 4: received 113652.81 pounds
Neighborhood 8: received 113911.46 pounds
Neighborhood 20: received 23547.32 pounds
Neighborhood 28: received 267239.25 pounds
Neighborhood 33: received 1.18393395e6 pounds
Neighborhood 40: received 40369.71 pounds
Neighborhood 50: received 105498.51 pounds
Neighborhood 56: received 979114.69 pounds
Neighborhood 59: received 23285.42 pounds
Neighborhood 80: received 238489.99 po

## Equity objective: minimizing worst unmet demand

We now consider an **equity-focused** objective that tries to avoid leaving any neighborhood with a very large unmet demand. Instead of minimizing total cost, we minimize the **maximum unmet demand** across neighborhoods.

### Sets

- $R$: set of restaurants, indexed by $i$
- $D$: set of food scrap centers, indexed by $j$
- $N$: set of neighborhoods, indexed by $k$

### Parameters

- $s_i$: supply (surplus food) at restaurant $i \in R$
- $d_k$: demand (food needed) in neighborhood $k \in N$

### Decision variables

- $x_{ij} \ge 0$: amount shipped from restaurant $i$ to donation center $j$
- $y_{jk} \ge 0$: amount shipped from donation center $j$ to neighborhood $k$
- $r_k$: food received by neighborhood $k$
- $t$: upper bound on unmet demand across all neighborhoods

### Equity optimization problem

We minimize the worst (largest) unmet demand:

$ \min \ t $

subject to:

$ \sum_{j \in D} x_{ij} \le s_i \quad \forall i \in R $  &nbsp; (restaurant supply)

$ \sum_{k \in N} y_{jk} = \sum_{i \in R} x_{ij} \quad \forall j \in D $  &nbsp; (flow conservation at centers)

$ r_k = \sum_{j \in D} y_{jk} \quad \forall k \in N $  &nbsp; (received by each neighborhood)

$ r_k \le d_k \quad \forall k \in N $  &nbsp; (cannot exceed demand)

$ t \ge d_k - r_k \quad \forall k \in N $  &nbsp; (unmet demand in each neighborhood)

$ x_{ij}, \, y_{jk} \ge 0 $

Interpretation:  
- $r_k$ captures how much food neighborhood $k$ actually receives._


In [10]:
# ================================
# Build index sets and parameters
# ================================

R = nrow(restaurant_data)          # number of restaurants
D = nrow(food_scrap_locations)     # number of food scrap centers
N = nrow(neighborhood_supply)      # number of neighborhoods

s = restaurant_data.supply         # s_i: supply at restaurant i
d = neighborhood_supply.demand     # d_k: demand at neighborhood k

# ================================
# Equity model: minimize worst unmet demand
# ================================
model_equity = Model(Gurobi.Optimizer)

set_silent(model_equity)

# Decision variables
@variable(model_equity, x[1:R, 1:D] >= 0)   # shipments restaurant -> center
@variable(model_equity, y[1:D, 1:N] >= 0)   # shipments center -> neighborhood
@variable(model_equity, r[1:N] >= 0)        # received by each neighborhood
@variable(model_equity, t >= 0)             # worst unmet demand across neighborhoods

# Objective: minimize worst unmet demand
@objective(model_equity, Min, t)

# 1) Restaurant supply: ∑_j x_ij ≤ s_i   ∀ i ∈ R
@constraint(model_equity, [i in 1:R],
    sum(x[i,j] for j in 1:D) <= s[i]
)

# 2) Flow conservation at centers:
#    ∑_k y_jk = ∑_i x_ij   ∀ j ∈ D
@constraint(model_equity, [j in 1:D],
    sum(y[j,k] for k in 1:N) == sum(x[i,j] for i in 1:R)
)

# 3) Neighborhood received:
#    r_k = ∑_j y_jk   ∀ k ∈ N
@constraint(model_equity, [k in 1:N],
    r[k] == sum(y[j,k] for j in 1:D)
)

# 4) Cannot exceed demand:
#    r_k ≤ d_k   ∀ k ∈ N
@constraint(model_equity, [k in 1:N],
    r[k] <= d[k]
)

# 5) Worst unmet demand:
#    t ≥ d_k - r_k   ∀ k ∈ N
@constraint(model_equity, [k in 1:N],
    t >= d[k] - r[k]
)

# ================================
# Solve and report
# ================================
optimize!(model_equity)

println("Equity model termination status: ", termination_status(model_equity))
println("Optimal worst unmet demand t*: ", value(t))

total_demand   = sum(d)
total_received = sum(value.(r))
total_unmet    = total_demand - total_received

println("Total demand:   ", total_demand)
println("Total received: ", total_received)
println("Total unmet:    ", total_unmet)
println("Demand reduction (%): ", 100 * total_received / total_demand, "%")


# Pull received amounts
r_vals = value.(r)

println("=== Neighborhoods That Received Food ===\n")

count_nonzero = 0
for k in 1:N
    if r_vals[k] > 0
        count_nonzero += 1
        println("Neighborhood $(k): received $(round(r_vals[k], digits=2)) units")
    end
end

println("\nTotal neighborhoods receiving any food: $count_nonzero")


Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Equity model termination status: OPTIMAL
Optimal worst unmet demand t*: 721313.1739676003
Total demand:   1.5243933021550444e8
Total received: 7.508486887683785e7
Total unmet:    7.735446133866659e7
Demand reduction (%): 49.25557516599548%
=== Neighborhoods That Received Food ===

Neighborhood 33: received 462620.77 units
Neighborhood 56: received 257801.52 units
Neighborhood 69: received 1.510081e6 units
Neighborhood 129: received 2.21059569e6 units
Neighborhood 140: received 780939.23 units
Neighborhood 141: received 1.3419113e6 units
Neighborhood 149: received 74288.42 units
Neighborhood 156: received 1.7813577e6 units
Neighborhood 162: received 487567.65 units
Neighborhood 165: received 1.44824261e6 units
Neighborhood 166: received 1.10169799e6 units
Neighborhood 168: received 1.37754376e6 units
Neighborhood 172: received 1.60768203e6 units
Neighborhoo

## Multi-Objective Optimization: Balancing Cost Reduction and Equity

We now combine the two objectives studied earlier:

1. **Reduced Cost Objective** — minimize total transportation cost plus a large penalty on unmet demand.  
2. **Equity Objective** — minimize the worst unmet demand across neighborhoods.

These two goals can conflict:  
efficient routing can increase inequity, while equitable distribution can increase transportation cost.

To explore the tradeoff, we form a **scalarized multi-objective**:

$$
\min \;
w_{\text{cost}} \cdot \text{CostObjective}
\;+\;
w_{\text{eq}} \cdot \text{EquityObjective}.
$$

### Cost objective component

$$
\text{CostObjective} =
\sum_{i \in R} \sum_{j \in D} c_{ij} x_{ij}
\;+\;
\sum_{j \in D} \sum_{k \in N} c_{jk} y_{jk}
\;+\;
M \sum_{k \in N} u_k,
$$

where $u_k$ represents unmet demand in neighborhood $k$.

### Equity objective component

The equity term is simply:

$$
\text{EquityObjective} = t,
$$

where $t$ is constrained so that:

$$
t \ge d_k - r_k \quad \forall k \in N.
$$

Thus, $t$ equals the **worst (maximum) unmet demand** across neighborhoods.

### Weight Sweep: $1{:}1$ through $1{:}10$ and back from $10{:}1$ to $2{:}1$

To fully explore the continuum between efficiency and fairness, we solve the combined model under **19 weight pairs**, starting with:

- $1{:}1, 1{:}2, \ldots, 1{:}10$  (equity increasingly emphasized)

and then sweeping back:

- $10{:}1, 9{:}1, \ldots, 2{:}1$  (cost increasingly emphasized)

Formally, the set of weight pairs is:

$$
(w_{\text{cost}}, w_{\text{eq}}) \in
\{(1,1), (1,2), \ldots, (1,10),
\; (10,1), (9,1), \ldots, (2,1)\}.
$$

For each pair, we record:

- total transportation cost  
- maximum unmet demand $t$  
- total food delivered  
- total unmet demand  
- which neighborhoods receive food  

This sweep traces out the **Pareto frontier** between cost reduction and equitable food distribution, revealing how solutions shift as we change the relative importance of the two objectives.


In [11]:
using JuMP
using Gurobi
using DataFrames

# Assume you already have:
# R, D, N
# supply (length R), demand (length N)
# cij[i,j], cjk[j,k]
# M defined (big-M for unmet demand)

function build_combined_model(w_cost::Float64, w_eq::Float64)
    model = Model(Gurobi.Optimizer)
    set_silent(model)

    # Decision variables
    @variable(model, x[1:R, 1:D] >= 0)   # restaurant -> center
    @variable(model, y[1:D, 1:N] >= 0)   # center -> neighborhood
    @variable(model, u[1:N] >= 0)        # unmet demand (for cost part)
    @variable(model, r[1:N] >= 0)        # received (for equity part)
    @variable(model, t >= 0)             # worst unmet demand

    # === Shared constraints ===

    # 1) Restaurant supply
    @constraint(model, [i in 1:R],
        sum(x[i,j] for j in 1:D) <= supply[i]
    )

    # 2) Flow conservation at centers
    @constraint(model, [j in 1:D],
        sum(y[j,k] for k in 1:N) == sum(x[i,j] for i in 1:R)
    )

    # 3) Demand balance for cost part:
    #    inflow + unmet = demand
    @constraint(model, [k in 1:N],
        sum(y[j,k] for j in 1:D) + u[k] == demand[k]
    )

    # 4) r_k = received = ∑_j y_jk
    @constraint(model, [k in 1:N],
        r[k] == sum(y[j,k] for j in 1:D)
    )

    # (optional, redundant but harmless) r_k ≤ d_k
    @constraint(model, [k in 1:N],
        r[k] <= demand[k]
    )

    # 5) Worst unmet demand: t ≥ d_k - r_k
    @constraint(model, [k in 1:N],
        t >= demand[k] - r[k]
    )

    # === Objective pieces ===

    @expression(model, cost_expr,
        sum(cij[i,j] * x[i,j] for i in 1:R, j in 1:D) +
        sum(cjk[j,k] * y[j,k] for j in 1:D, k in 1:N) +
        M * sum(u[k] for k in 1:N)
    )

    @expression(model, equity_expr, t)

    # Weighted objective: w_cost * cost + w_eq * equity
    @objective(model, Min, w_cost * cost_expr + w_eq * equity_expr)

    return model, x, y, u, r, t, cost_expr, equity_expr
end


build_combined_model (generic function with 1 method)

In [12]:
results = DataFrame(
    w_cost      = Float64[],
    w_eq        = Float64[],
    obj_value   = Float64[],
    cost_value  = Float64[],
    equity_t    = Float64[],
    total_recv  = Float64[],
    total_unmet = Float64[],
)

# 1:1, 1:2, ..., 1:10
for w_eq_int in 1:10
    w_cost = 1.0
    w_eq   = float(w_eq_int)   # convert Int -> Float64

    model, x, y, u, r, t, cost_expr, equity_expr = build_combined_model(w_cost, w_eq)
    optimize!(model)

    if termination_status(model) != MOI.OPTIMAL
        println("Weights (", w_cost, ", ", w_eq, ") not optimal, status = ", termination_status(model))
        continue
    end

    obj_val    = objective_value(model)
    cost_val   = value(cost_expr)
    t_val      = value(t)
    r_vals     = value.(r)
    u_vals     = value.(u)
    total_d    = sum(demand)
    total_recv = sum(r_vals)
    total_unm  = total_d - total_recv

    # transport cost (miles) = cost objective minus big-M penalty
    transport_cost = cost_val - M * sum(u_vals)

    # per-neighborhood remaining deficit
    unmet_by_k = demand .- r_vals
    max_unmet, worst_k = findmax(unmet_by_k)

    push!(results, (w_cost, w_eq, obj_val, cost_val, t_val, total_recv, total_unm))

    # === Print summary for this weight pair ===
    println("\n=== Weights: w_cost = $(w_cost), w_eq = $(w_eq) ===")
    println("Total miles driven (transport cost, no penalty): ",
            round(transport_cost, digits = 2))
    println("Worst remaining deficit: ",
            round(max_unmet, digits = 2),
            " units in neighborhood ", worst_k)

    println("\nNeighborhoods that received food:\n")

    count_nonzero = 0
    for k in 1:N
        if r_vals[k] > 1e-6  # small threshold to ignore numerical noise
            count_nonzero += 1
            println("  Neighborhood $(k): $(round(r_vals[k], digits=2)) units")
        end
    end

    println("\nTotal neighborhoods receiving any food: $count_nonzero\n")
end

# 10:1, 9:1, ..., 2:1
for w_cost_int in 10:-1:2
    w_eq   = 1.0
    w_cost = float(w_cost_int)   # convert Int -> Float64

    model, x, y, u, r, t, cost_expr, equity_expr = build_combined_model(w_cost, w_eq)
    optimize!(model)

    if termination_status(model) != MOI.OPTIMAL
        println("Weights (", w_cost, ", ", w_eq, ") not optimal, status = ", termination_status(model))
        continue
    end

    obj_val    = objective_value(model)
    cost_val   = value(cost_expr)
    t_val      = value(t)
    r_vals     = value.(r)
    u_vals     = value.(u)
    total_d    = sum(demand)
    total_recv = sum(r_vals)
    total_unm  = total_d - total_recv

    # transport cost (miles) = cost objective minus big-M penalty
    transport_cost = cost_val - M * sum(u_vals)

    # per-neighborhood remaining deficit
    unmet_by_k = demand .- r_vals
    max_unmet, worst_k = findmax(unmet_by_k)

    push!(results, (w_cost, w_eq, obj_val, cost_val, t_val, total_recv, total_unm))

    # === Print summary for this weight pair ===
    println("\n=== Weights: w_cost = $(w_cost), w_eq = $(w_eq) ===")
    println("Total miles driven (transport cost, no penalty): ",
            round(transport_cost, digits = 2))
    println("Worst remaining deficit: ",
            round(max_unmet, digits = 2),
            " units in neighborhood ", worst_k)

    println("\nNeighborhoods that received food:\n")

    count_nonzero = 0
    for k in 1:N
        if r_vals[k] > 1e-6
            count_nonzero += 1
            println("  Neighborhood $(k): $(round(r_vals[k], digits=2)) units")
        end
    end

    println("\nTotal neighborhoods receiving any food: $count_nonzero\n")
end

println("=== Summary table over all weights ===")
println(results)


Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03

=== Weights: w_cost = 1.0, w_eq = 1.0 ===
Total miles driven (transport cost, no penalty): 7.373734e6
Worst remaining deficit: 2.01461008e6 units in neighborhood 250

Neighborhoods that received food:

  Neighborhood 1: 102142.62 units
  Neighborhood 2: 333492.65 units
  Neighborhood 4: 113652.81 units
  Neighborhood 8: 113911.46 units
  Neighborhood 20: 23547.32 units
  Neighborhood 28: 267239.25 units
  Neighborhood 33: 1.18393395e6 units
  Neighborhood 40: 40369.71 units
  Neighborhood 50: 105498.51 units
  Neighborhood 59: 23285.42 units
  Neighborhood 69: 216784.09 units
  Neighborhood 80: 238489.99 units
  Neighborhood 93: 89959.74 units
  Neighborhood 101: 704873.88 units
  Neighborhood 111: 258687.53 units
  Neighborhood 114: 16549.04 units
  Neighborhood 118: 261754.51 units
  Neighborhood 120: 405519.22 units
  Neighborhood 129: 917298.78 units


In [13]:
results = DataFrame(
    w_cost      = Float64[],
    w_eq        = Float64[],
    obj_value   = Float64[],
    cost_value  = Float64[],
    equity_t    = Float64[],
    total_recv  = Float64[],
    total_unmet = Float64[],
)

# 1:1, 1:2, ..., 1:10
for w_eq_int in 1:10
    w_cost = 1.0
    w_eq   = float(w_eq_int)

    model, x, y, u, r, t, cost_expr, equity_expr = build_combined_model(w_cost, w_eq)
    optimize!(model)

    if termination_status(model) != MOI.OPTIMAL
        println("Weights (", w_cost, ", ", w_eq, ") not optimal, status = ", termination_status(model))
        continue
    end

    obj_val    = objective_value(model)
    cost_val   = value(cost_expr)
    t_val      = value(t)
    r_vals     = value.(r)
    u_vals     = value.(u)
    total_d    = sum(demand)
    total_recv = sum(r_vals)
    total_unm  = total_d - total_recv

    transport_cost = cost_val - M * sum(u_vals)
    unmet_by_k = demand .- r_vals
    max_unmet, worst_k = findmax(unmet_by_k)

    push!(results, (w_cost, w_eq, obj_val, cost_val, t_val, total_recv, total_unm))

    println("\n=== Weights: w_cost = $(w_cost), w_eq = $(w_eq) ===")
    println("Total miles driven (transport cost, no penalty): ",
            round(transport_cost, digits = 2))
    println("Worst remaining deficit: ",
            round(max_unmet, digits = 2),
            " units in neighborhood ", worst_k)

    println("Total food given away: ",
               round(total_recv, digits = 2), " units")

    println("\nNeighborhoods that received food:\n")

    count_nonzero = 0
    for k in 1:N
        if r_vals[k] > 1e-6
            count_nonzero += 1
            println("  Neighborhood $(k): $(round(r_vals[k], digits=2)) units")
        end
    end

    println("\nTotal neighborhoods receiving any food: $count_nonzero\n")
end

# 10:1, 9:1, ..., 2:1
for w_cost_int in 10:-1:2
    w_eq   = 1.0
    w_cost = float(w_cost_int)

    model, x, y, u, r, t, cost_expr, equity_expr = build_combined_model(w_cost, w_eq)
    optimize!(model)

    if termination_status(model) != MOI.OPTIMAL
        println("Weights (", w_cost, ", ", w_eq, ") not optimal, status = ", termination_status(model))
        continue
    end

    obj_val    = objective_value(model)
    cost_val   = value(cost_expr)
    t_val      = value(t)
    r_vals     = value.(r)
    u_vals     = value.(u)
    total_d    = sum(demand)
    total_recv = sum(r_vals)
    total_unm  = total_d - total_recv

    transport_cost = cost_val - M * sum(u_vals)
    unmet_by_k = demand .- r_vals
    max_unmet, worst_k = findmax(unmet_by_k)

    push!(results, (w_cost, w_eq, obj_val, cost_val, t_val, total_recv, total_unm))

    println("\n=== Weights: w_cost = $(w_cost), w_eq = $(w_eq) ===")
    println("Total miles driven (transport cost, no penalty): ",
            round(transport_cost, digits = 2))
    println("Worst remaining deficit: ",
            round(max_unmet, digits = 2),
            " units in neighborhood ", worst_k)

    println("Total food given away: ",
               round(total_recv, digits = 2), " units")

    println("\nNeighborhoods that received food:\n")

    count_nonzero = 0
    for k in 1:N
        if r_vals[k] > 1e-6
            count_nonzero += 1
            println("  Neighborhood $(k): $(round(r_vals[k], digits=2)) units")
        end
    end

    println("\nTotal neighborhoods receiving any food: $count_nonzero\n")
end

println("=== Summary table over all weights ===")
println(results)


Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03

=== Weights: w_cost = 1.0, w_eq = 1.0 ===
Total miles driven (transport cost, no penalty): 7.373734e6
Worst remaining deficit: 2.01461008e6 units in neighborhood 250
Total food given away: 7.508486888e7 units

Neighborhoods that received food:

  Neighborhood 1: 102142.62 units
  Neighborhood 2: 333492.65 units
  Neighborhood 4: 113652.81 units
  Neighborhood 8: 113911.46 units
  Neighborhood 20: 23547.32 units
  Neighborhood 28: 267239.25 units
  Neighborhood 33: 1.18393395e6 units
  Neighborhood 40: 40369.71 units
  Neighborhood 50: 105498.51 units
  Neighborhood 59: 23285.42 units
  Neighborhood 69: 216784.09 units
  Neighborhood 80: 238489.99 units
  Neighborhood 93: 89959.74 units
  Neighborhood 101: 704873.88 units
  Neighborhood 111: 258687.53 units
  Neighborhood 114: 16549.04 units
  Neighborhood 118: 261754.51 units
  Neighborhood 120: 405519.22

## Stochastic Demand

## Parameters in Formulation

- $R$: Set of restaurants (supply nodes).
- $D$: Set of candidate donation or transshipment centers.
- $N$: Set of neighborhoods (demand nodes).
- $S$: Set of demand scenarios.
- $a_i$: Surplus food available at restaurant $i \in R$.
- $d_k^s$: Demand at neighborhood $k \in N$ under scenario $s \in S$.
- $c_{ij}$: Transportation cost (or distance) from restaurant $i$ to center $j$.
- $c_{jk}$: Transportation cost from center $j$ to neighborhood $k$.
- $F_j$: Fixed facility-opening cost for center $j \in D$.
- $p_s$: Probability of scenario $s \in S$.
- $M$: Penalty coefficient for unmet demand.
- $\overline{Q}_j$: Capacity parameter linking flow through center $j$ to the binary build decision $z_j$ (big-$M$ type).

---

## Formulation: Stochastic Minimum Distance

$$
\min \quad \sum_{j \in D} F_j z_j + \sum_{s \in S} p_s \left( \sum_{i \in R} \sum_{j \in D} c_{ij} x_{ij}^s + \sum_{j \in D} \sum_{k \in N} c_{jk} y_{jk}^s + M \sum_{k \in N} u_k^s \right)
$$

**Subject to:**

$$
\sum_{j \in D} x_{ij}^s \leq a_i \qquad \forall i \in R, \; \forall s \in S
$$

$$
\sum_{j \in D} y_{jk}^s + u_k^s = d_k^s \qquad \forall k \in N, \; \forall s \in S
$$

$$
\sum_{k \in N} y_{jk}^s = \sum_{i \in R} x_{ij}^s \qquad \forall j \in D, \; \forall s \in S
$$

$$
\sum_{i \in R} x_{ij}^s \leq \overline{Q}_j z_j \qquad \forall j \in D, \; \forall s \in S
$$

$$
z_j \in \{0, 1\} \qquad \forall j \in D
$$

$$
x_{ij}^s, \; y_{jk}^s, \; u_k^s \geq 0 \qquad \forall i \in R, \; \forall j \in D, \; \forall k \in N, \; \forall s \in S
$$

---

## Interpretation of Formulation

Formulation 1 describes a **two-stage stochastic minimum-distance model** in which the planner chooses which donation centers to open (via the binary variables $z_j$) before uncertainty in demand is realized, and then determines optimal food flows for each scenario $s \in S$.

The **objective** minimizes the total expected transportation cost (cost from restaurants to donation centers should probs be equal to cost from donation centers to neighborhoods) plus any unmet-demand penalties, together with fixed opening costs.

**Constraints:**

1. **First constraint:** Ensures that each restaurant $i$ can send at most its available surplus $a_i$.

2. **Second constraint:** States that, for each center $j$, the total inflow from restaurants must equal its total outflow to neighborhoods.

3. **Third constraint:** Enforces that each neighborhood $k$ in scenario $s$ receives food $r_k^s$ equal to the flow from all centers plus any unmet demand $u_k^s$ must satisfy total demand $d_k^s$.

4. **Fourth constraint:** Links the flows into each center $j$ to its opening decision through the capacity parameter $\overline{Q}_j$, ensuring no flow can pass through a center that is not opened.

In [ ]:
R = nrow(restaurant_data)
D = nrow(food_scrap_locations)
N = nrow(neighborhood_supply[neighborhood_supply.Year .== 2023, :])
Q = sum(restaurant_data.supply)

M = sum(demand)
F = 10000       # Center opening cost

S = 3 # 3 demand years

demand_2023 = neighborhood_supply[neighborhood_supply.Year .== 2023, :].demand
demand_2024 = neighborhood_supply[neighborhood_supply.Year .== 2024, :].demand
demand_2025 = neighborhood_supply[neighborhood_supply.Year .== 2025, :].demand

demand_df = DataFrame(
    y2023 = demand_2023,
    y2024 = demand_2024,
    y2025 = demand_2025
)

Row,y2023,y2024,y2025
,Float64,Float64,Float64
1,0.0,0.0,1.02143e5
2,0.0,0.0,3.33493e5
3,0.0,0.0,0.0
4,0.0,0.0,1.13653e5
5,0.0,0.0,0.0
6,82217.3,13128.3,5.06273e5
7,2.25651e5,0.0,0.0
8,9.04474e5,0.0,1.13911e5
9,0.0,0.0,0.0


In [ ]:
println("Total supply: ", sum(supply))
println("Total demand (avg): ", mean([sum(demand_df[:,s]) for s in 1:S]))
println("Supply/Demand ratio: ", sum(supply) / mean([sum(demand_df[:,s]) for s in 1:S]))

Total supply: 7.508486887683786e7
Total demand (avg): 5.081311007183481e7
Supply/Demand ratio: 1.4776672549798648


In [ ]:
# Assume you already have:
# R, D, N
# supply (length R), demand (length N)
# cij[i,j], cjk[j,k]
# M defined (big-M for unmet demand)

function build_combined_model(w_cost::Float64, w_eq::Float64)
    model = Model(Gurobi.Optimizer)
    set_silent(model)

    @variable(model, x[s=1:S, 1:R, 1:D] >= 0)   # restaurant -> center
    @variable(model, y[s=1:S, 1:D, 1:N] >= 0)   # center -> neighborhood
    @variable(model, u[s=1:S, 1:N] >= 0)        # unmet demand
    @variable(model, r[s=1:S, 1:N] >= 0)        # received
    @variable(model, t[s=1:S] >= 0)             # worst unmet demand
    @variable(model, z[1:D], Bin)               # center open (first stage)

    # Restaurant supply (optional: could be scenario-dependent if supply changes)
    @constraint(model, [s in 1:S, i in 1:R],
        sum(x[s,i,j] for j in 1:D) <= supply[i]
    )
    
    # Flow conservation at centers
    @constraint(model, [s in 1:S, j in 1:D],
        sum(y[s,j,k] for k in 1:N) == sum(x[s,i,j] for i in 1:R)
    )
    
    # Demand satisfaction
    @constraint(model, [s in 1:S, k in 1:N],
        sum(y[s,j,k] for j in 1:D) + u[s,k] == demand_df[k,s]
    )
    
    # r_k = received
    @constraint(model, [s in 1:S, k in 1:N],
        r[s,k] == sum(y[s,j,k] for j in 1:D)
    )
    
    # r_k ≤ demand
    @constraint(model, [s in 1:S, k in 1:N],
        r[s,k] <= demand_df[k,s]
    )
    
    # Worst unmet demand
    @constraint(model, [s in 1:S, k in 1:N],
        t[s] >= demand_df[k,s] - r[s,k]
    )
    
    # Center capacity constraint (link to first-stage z)
    @constraint(model, [s in 1:S, j in 1:D],
        sum(x[s,i,j] for i in 1:R) <= Q * z[j]
    )


    # === Objective pieces ===
    @objective(model, Min,
        F * sum(z[j] for j in 1:D) +
        sum(1/3 *
            w_cost * (
                sum(cij[i,j] * x[s,i,j] for i in 1:R, j in 1:D) +
                sum(cjk[j,k] * y[s,j,k] for j in 1:D, k in 1:N) +
                M * sum(u[k] for k in 1:N)
                ) +
            w_eq * (
                t[s]
                )
            for s in 1:S)
        )

    return model, x, y, u, r, t, z, w_cost, w_eq
end


build_combined_model (generic function with 1 method)

In [18]:
model, x, y, u, r, t, z, cost_expr, equity_expr = build_combined_model(1.0, 1.0)
optimize!(model)

obj_val    = objective_value(model)


: 

: 

In [26]:
# Solve the model
model, x, y, u, r, t, cost_expr, equity_expr = build_combined_model(1.0, 1.0)
optimize!(model)

println("=== Summary per Scenario ===")
for s in 1:S
    total_flow_x = sum(value.(x[s, :, :]))
    total_flow_y = sum(value.(y[s, :, :]))
    total_unmet = sum(value.(u[s, :]))
    worst_unmet = value(t[s])
    
    println("Scenario $s:")
    println("  Total flow from restaurants to centers: ", total_flow_x)
    println("  Total flow from centers to neighborhoods: ", total_flow_y)
    println("  Total unmet demand: ", total_unmet)
    println("  Worst unmet demand: ", worst_unmet)
    println("------------------------------------------------")
end

# Total number of centers opened (same for all scenarios)
num_centers_open = sum(value.(z))
println("Total number of centers opened: ", num_centers_open)


Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
=== Summary per Scenario ===
Scenario 1:
  Total flow from restaurants to centers: 1.6034409875006376e7
  Total flow from centers to neighborhoods: 1.6034409875006378e7
  Total unmet demand: 2.6580457696113624e7
  Worst unmet demand: 2.882998136e6
------------------------------------------------
Scenario 2:
  Total flow from restaurants to centers: 1.6034409875006378e7
  Total flow from centers to neighborhoods: 1.6034409875006376e7
  Total unmet demand: 4.303078815641451e7
  Worst unmet demand: 3.62417679513963e6
------------------------------------------------
Scenario 3:
  Total flow from restaurants to centers: 1.5716457907803938e7
  Total flow from centers to neighborhoods: 1.5716457907803936e7
  Total unmet demand: 3.504280670515962e7
  Worst unmet demand: 1.5469672167837997e6
------------------------------------------------
Total number of centers o

# Extract the data needed for visualization

In [ ]:
# ============================================================================
# 1. Generate Pareto Frontier (optimized - fewer redundant points)
# ============================================================================
results = DataFrame(
    w_cost = Float64[],
    w_eq = Float64[],
    obj_value = Float64[],
    num_centers = Int[],
    avg_transport_cost = Float64[],
    avg_equity_t = Float64[],
    avg_total_recv = Float64[],
    avg_total_unmet = Float64[],
    t_2023 = Float64[],
    t_2024 = Float64[],
    t_2025 = Float64[]
)

# Streamlined weight pairs (~35 instead of ~100)
# weight_pairs = vcat(
#     [(1.0, w) for w in [0.5, 1.0, 1.5, 2.0, 3.0, 5.0, 8.0, 15.0, 25.0]],  # 9 pts
#     [(w, 1.0) for w in [2.0, 3.0, 5.0, 8.0]],                              # 4 pts
#     [(1.0, w) for w in 4.0:1.0:7.0],                                       # 4 pts - target gap
#     [(2.0, w) for w in 2.0:1.0:6.0],                                       # 5 pts - target gap
#     [(3.0, w) for w in 2.0:1.0:5.0],                                       # 4 pts - target gap
#     [(w, w) for w in [1.5, 2.0, 2.5, 3.0, 4.0]],                          # 5 pts - diagonal
#     [(1.5, w) for w in [3.0, 4.0, 5.0, 6.0]]                              # 4 pts - target gap
# )

# Minimal weight pairs for testing (~10 points)
weight_pairs = [
    (1.0, 0.5),   # cost-focused
    (1.0, 1.0),   # balanced
    (1.0, 2.0),
    (1.0, 5.0),
    (1.0, 10.0),
    (1.0, 20.0),  # equity-focused
    (2.0, 1.0),
    (5.0, 1.0),
    (1.0, 3.0),
    (1.0, 7.0)
]

# Remove duplicates
weight_pairs = unique(weight_pairs)

for (w_cost, w_eq) in weight_pairs
    model, x, y, u, r, t, z, _, _ = build_combined_model(w_cost, w_eq)
    optimize!(model)
    
    termination_status(model) != MOI.OPTIMAL && continue
    
    num_centers = Int(round(sum(value.(z))))
    
    transport_costs = [sum(cij[i,j] * value(x[s,i,j]) for i in 1:R, j in 1:D) +
                       sum(cjk[j,k] * value(y[s,j,k]) for j in 1:D, k in 1:N) for s in 1:S]
    
    total_recvs = [sum(value.(r[s,:])) for s in 1:S]
    total_unmets = [sum(value.(u[s,:])) for s in 1:S]
    equity_ts = [value(t[s]) for s in 1:S]
    
    push!(results, (
        w_cost, w_eq,
        objective_value(model),
        num_centers,
        mean(transport_costs),
        mean(equity_ts),
        mean(total_recvs),
        mean(total_unmets),
        equity_ts[1], equity_ts[2], equity_ts[3]
    ))
end

CSV.write("viz_data/pareto_results.csv", results)

Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic lic

"viz_data/pareto_results.csv"

In [20]:
# ============================================================================
# 2. Export Location Data
# ============================================================================
CSV.write("viz_data/restaurants.csv", DataFrame(
    id = 1:R,
    latitude = restaurant_data.latitude,
    longitude = restaurant_data.longitude,
    supply = restaurant_data.supply
))

CSV.write("viz_data/donation_centers.csv", DataFrame(
    id = 1:D,
    latitude = food_scrap_locations.latitude,
    longitude = food_scrap_locations.longitude
))

# Update the neighborhoods export to include names
# Reload original neighborhood data to get names
neighborhood_full = CSV.read(neighborhood_path, DataFrame)
nbhd_2023_full = neighborhood_full[neighborhood_full.Year .== 2023, :]

# Export neighborhoods with names
CSV.write("viz_data/neighborhoods.csv", DataFrame(
    id = 1:N,
    name = nbhd_2023_full[!, "Neighborhood.Tabulation.Area..NTA..Name"],
    latitude = nbhd_2023_full.latitude,
    longitude = nbhd_2023_full.longitude,
    demand_2023 = demand_df.y2023,
    demand_2024 = demand_df.y2024,
    demand_2025 = demand_df.y2025
))

# ============================================================================
# 3. Export Scenario-Specific Results for a Balanced Solution
# ============================================================================
model_bal, x_bal, y_bal, u_bal, r_bal, t_bal, z_bal, _, _ = build_combined_model(1.0, 3.0)
optimize!(model_bal)

if termination_status(model_bal) == MOI.OPTIMAL
    # Centers opened
    CSV.write("viz_data/centers_opened.csv", DataFrame(
        id = 1:D,
        opened = Int.(round.(value.(z_bal)))
    ))
    
    # Per-scenario allocations
    for s in 1:S
        year = 2022 + s
        CSV.write("viz_data/allocations_$year.csv", DataFrame(
            neighborhood_id = 1:N,
            received = value.(r_bal[s,:]),
            unmet = value.(u_bal[s,:])
        ))
    end
    
    # Flows for scenario 1 (2023)
    flows = DataFrame(from_type=String[], from_id=Int[], to_type=String[], to_id=Int[], flow=Float64[])
    for i in 1:R, j in 1:D
        value(x_bal[1,i,j]) > 1e-3 && push!(flows, ("restaurant", i, "center", j, value(x_bal[1,i,j])))
    end
    for j in 1:D, k in 1:N
        value(y_bal[1,j,k]) > 1e-3 && push!(flows, ("center", j, "neighborhood", k, value(y_bal[1,j,k])))
    end
    CSV.write("viz_data/flows_2023.csv", flows)
end

# ============================================================================
# 4. Export Scenario Comparison Data
# ============================================================================
scenario_summary = DataFrame(
    scenario = ["2023", "2024", "2025"],
    total_demand = [sum(demand_df.y2023), sum(demand_df.y2024), sum(demand_df.y2025)],
    total_received = [sum(value.(r_bal[s,:])) for s in 1:S],
    total_unmet = [sum(value.(u_bal[s,:])) for s in 1:S],
    worst_unmet = [value(t_bal[s]) for s in 1:S]
)
CSV.write("viz_data/scenario_summary.csv", scenario_summary)

Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03


"viz_data/scenario_summary.csv"

In [21]:
# ============================================================================
# 5. Export Allocations for ALL Pareto Solutions (for animation)
# ============================================================================
mkpath("viz_data/pareto_allocations")

for (idx, (w_cost, w_eq)) in enumerate(weight_pairs)
    model, x, y, u, r, t, z, _, _ = build_combined_model(w_cost, w_eq)
    optimize!(model)
    
    termination_status(model) != MOI.OPTIMAL && continue
    
    # Average unmet across scenarios
    avg_unmet = [mean([value(u[s,k]) for s in 1:S]) for k in 1:N]
    
    CSV.write("viz_data/pareto_allocations/alloc_$(idx).csv", DataFrame(
        neighborhood_id = 1:N,
        avg_unmet = avg_unmet
    ))
end

println("Exported $(length(weight_pairs)) allocation files")

Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic license - for non-commercial use only - expires 2026-09-03
Set parameter Username
Set parameter LicenseID to value 2702779
Academic lic